In [1]:
import os
import random
from typing import Tuple

import numpy as np
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.backends import cudnn
from torch.nn import functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset, random_split
from tqdm import tqdm
from torch.autograd import Variable
torch.set_printoptions(precision=4)


def set_seed(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

set_seed(1234)

In [2]:
latent_img_train       = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_train.npy')
latent_img_label_train = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_label_train.npy')

latent_img_test        = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_test.npy')
latent_img_label_test  = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_img_label_test.npy')

latent_sensor_corrupted_train   = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_corrupted_train.npy')
latent_sensor_uncorrupted_train = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_uncorrupted_train.npy')
latent_sensor_label_train       = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_label_train.npy')

latent_sensor_corrupted_test    = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_corrupted_test.npy')
latent_sensor_uncorrupted_test  = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_uncorrupted_test.npy')
latent_sensor_label_test        = np.load('/home/ubuntu/Latent-Transfer/Latent-to-Latent-overfit/Latent_Spaces/latent_sensor_label_test.npy')

In [7]:
def create_dataset(
    batch_size = 32,
    split      = 0.8,
    seed       = 1234,
    img_train  = latent_img_train , 
    img_train_label    = latent_img_label_train,
    sensor_train       = latent_sensor_uncorrupted_train,
    sensor_train_label = latent_sensor_label_train
):
    np.random.seed(seed)

    x_img    = np.array(img_train) 
    x_sensor = np.array(sensor_train) 
    y_img    = np.array(img_train_label)
    y_sensor = np.array(sensor_train_label)

    # x_img    = np.array(img_train)[0]
    # x_sensor = np.array(sensor_train)[0]
    # y_img    = np.array(img_train_label)[0]
    # y_sensor = np.array(sensor_train_label)[0]
   
    tensor_x_img    = torch.Tensor(x_img)
    tensor_x_sensor = torch.Tensor(x_sensor)
    tensor_y_img    = torch.Tensor(y_img)
    tensor_y_sensor = torch.Tensor(y_sensor)

    # train_split = int(split * len(x_img))
    # val_split   = int(len(x_img) - train_split)
    # valid_split = int(val_split / 2)
    # test_split  = int(val_split - valid_split)
   
    tensor_dataset = TensorDataset(tensor_x_img, tensor_x_sensor, tensor_y_img, tensor_y_sensor)

    # train_dataset, val_dataset = random_split(tensor_dataset, [train_split, val_split])
    # val_dataset, test_dataset  = random_split(val_dataset, [valid_split, test_split])

    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    train_loader = DataLoader(tensor_dataset, batch_size=batch_size, shuffle=True)

    return train_loader

In [8]:
class Latent_Encoder(nn.Module):
    def __init__(self):
        super(Latent_Encoder, self).__init__()
        self.layer1 = nn.Linear(256, 256)
        self.dropout1 = nn.Dropout(0.45)
        self.layer2 = nn.Linear(256, 256)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.layer2(x)
        
        return x

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Latent_Encoder().cuda()
train_loader = create_dataset()
# min_val_img, max_val_img, min_val_sensor, max_val_sensor = get_min_and_max(train_loader, device)

In [10]:
len(train_loader), device

(858, device(type='cuda'))

In [11]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

In [12]:
for epoch in range(100):
  train_loss = 0
  for data in tqdm(train_loader):
  
      x_img, x_sensor, y_img, y_sensor = data

      x_img = x_img.to(device)
      x_sensor = x_sensor.to(device)

      optimizer.zero_grad()
      y = model(x_sensor)
      loss = F.mse_loss(x_img, y)
 
      loss.backward()
      optimizer.step()
      train_loss += loss.item()
      
  # if(epoch%100 == 0):
  scheduler.step(train_loss/len(train_loader))
  print("Epoch: {} || Train Loss: {}".format(epoch + 1, train_loss / len(train_loader)))

100%|██████████| 858/858 [00:04<00:00, 196.75it/s]


Epoch: 1 || Train Loss: 0.9963764856745313


100%|██████████| 858/858 [00:02<00:00, 306.67it/s]


Epoch: 2 || Train Loss: 0.9942540499297056


100%|██████████| 858/858 [00:02<00:00, 286.92it/s]


...

Epoch: 7 || Train Loss: 0.9937856744775128


100%|██████████| 858/858 [00:02<00:00, 300.66it/s]


Epoch: 8 || Train Loss: 0.9938982727227511


 92%|█████████▏| 790/858 [00:02<00:00, 297.90it/s]


KeyboardInterrupt: 

In [14]:
x = train_loader.dataset[0][0].cuda()
y = train_loader.dataset[0][1].cuda()

In [15]:
model.eval()

Latent_Encoder(
  (layer1): Linear(in_features=256, out_features=256, bias=True)
  (dropout1): Dropout(p=0.25, inplace=False)
  (layer2): Linear(in_features=256, out_features=256, bias=True)
)

In [16]:
yy = model(y)

In [17]:
y[0,:10], yy[0,:10], x[0,:10]

(tensor([ 0.7688,  0.0577,  0.0748, -0.0028,  0.2359, -0.0951,  0.2604,  0.3080,
         -0.0952,  0.0082], device='cuda:0'),
 tensor([ 6.2906e-05, -4.5801e-02, -2.9159e-02,  1.5213e-02, -7.4636e-02,
          2.8482e-02,  7.3000e-03,  4.3644e-02,  3.1700e-02, -3.0197e-02],
        device='cuda:0', grad_fn=<SliceBackward>),
 tensor([-0.5153,  0.2756, -0.5895, -0.6621, -1.1936, -0.3752, -1.0706,  1.3290,
          1.9911,  1.8511], device='cuda:0'))